In [0]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
import pandas as pd
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from gensim.models.doc2vec import Doc2Vec

In [0]:
downloaded = drive.CreateFile({'id':'14kt0EB7K36gd5k5JQ4-752Cl-BPZ-3tl'}) ## Load Secondary_Source.csv in the edge_list_creation folder
downloaded.GetContentFile('2_col_earl_samp.json')  
file_dictionary = pd.read_json('2_col_earl_samp.json')

In [4]:
file_dictionary

,Description,Opinion
0,"LABOR BOARD v. SEVEN-UP CO., 344 U.S. 344 (1953)",\n\nUnited States Supreme Court\nLABOR BOARD v...
1,"EDELMAN v. CALIFORNIA, 344 U.S. 357 (1953)",\n\nUnited States Supreme Court\nEDELMAN v. CA...
2,"PENNSYLVANIA R. CO. v. O'ROURKE, 344 U.S. 334 ...",\n\nUnited States Supreme Court\nPENNSYLVANIA ...
3,"NEW YORK v. N. Y., N. H. & H. R. CO., 344 U.S....",\n\nUnited States Supreme Court\nNEW YORK v. N...
4,"SOUTH BUFFALO R. CO. v. AHERN, 344 U.S. 367 (1...",\n\nUnited States Supreme Court\nSOUTH BUFFALO...
...,...,...
3225,"OESTEREICH v. SELECTIVE SERVICE BD., 393 U.S. ...",\n\nUnited States Supreme Court\nOESTEREICH v....
3226,"WILSON v. KELLEY, 393 U.S. 266 (1968)",\n\nUnited States Supreme Court\nWILSON v. KEL...
3227,"JOHNSON v. BENNETT, 393 U.S. 253 (1968)",\n\nUnited States Supreme Court\nJOHNSON v. BE...
3228,"CLARK v. GABRIEL, 393 U.S. 256 (1968)",\n\nUnited States Supreme Court\nCLARK v. GABR...


In [0]:
file_dictionary = file_dictionary.sample(1000)

In [6]:
file_dictionary

,Description,Opinion
1515,FAIRVIEW PUBLIC UTILITY DIST. NO. 1 v. CITY OF...,\n\nUnited States Supreme Court\nFULGHUM v. LO...
1008,CERMINARO v. URBAN REDEVELOPMENT AUTH. OF PITT...,\n\nUnited States Supreme Court\nCERMINARO v. ...
602,"YATES v. UNITED STATES, 356 U.S. 363 (1958)",\n\nUnited States Supreme Court\nYATES v. UNIT...
1211,"TEAMSTERS LOCAL v. LABOR BOARD, 365 U.S. 667 (...",\n\nUnited States Supreme Court\nTEAMSTERS LOC...
26,"LABOR BOARD v. GAMBLE ENTERPRISES, 345 U.S. 11...",\n\nUnited States Supreme Court\nLABOR BOARD v...
...,...,...
595,"DENVER STOCK YARD v. LIVESTOCK ASSN., 356 U.S....",\n\nUnited States Supreme Court\nDENVER STOCK ...
393,"MEAT CUTTERS v. FAIRLAWN MEATS, 353 U.S. 20 (1...",\n\nUnited States Supreme Court\nMEAT CUTTERS ...
430,"BRITISH TRANSPORT COMM'N v. US, 354 U.S. 129 (...",\n\nUnited States Supreme Court\nBRITISH TRANS...
2335,"WALKER, INC. v. FOOD MACHINERY, 382 U.S. 172 (...","\n\nUnited States Supreme Court\nWALKER, INC. ..."


In [0]:
file = file_dictionary.values

In [8]:
import nltk
from nltk.corpus import stopwords
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [0]:
### Get the stopwords from different languages from nltk package
# french_stopwords = stopwords.words('french')
# german_stopwords = stopwords.words('german')
# turkish_stopwords = stopwords.words('turkish')

In [0]:
## This is a function designed to remove all the stopwords in our text
def preprocess(text): # one little problem, gensim.utils.simple_preprocess gets rid off "I" automatically. 
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
                result.append(token)
    return result

In [0]:
file_dictionary = {}
for i in file:
  file_dictionary[i[0]] = i[1]

In [0]:
for key in file_dictionary:
  #print("working on file "+ key)
  file_dictionary[key] = preprocess(file_dictionary[key])

In [0]:
##this function tag all processed files so it is the form for Doc2Vec model 
def tag_doc(processed, i):
  return gensim.models.doc2vec.TaggedDocument(processed, [str(i)])

In [0]:
temp_a = [(k,v) for k,v in file_dictionary.items()]

In [0]:
temp = [i[1] for i in temp_a]

In [0]:
processed_files = []
for i in range(len(temp)):
  processed_files.append(tag_doc(temp[i], i))

In [0]:
test_files = []
for i in range(len(temp)):
  test_files.append(temp[i])

In [0]:
model = gensim.models.doc2vec.Doc2Vec(vector_size=50, min_count=1, epochs=40)

In [0]:
model.build_vocab(processed_files)

In [0]:
model.train(processed_files, total_examples=model.corpus_count, epochs=model.epochs)

In [0]:
threshold = 0.9 #
Source = []
Target = []
Weight = []
number_of_doc = 1000 ##change it 1346 for all

In [2]:
for i in range(number_of_doc // 2):
  print("working on " + str(i))
  for k in range(i+1, number_of_doc // 2):    
    similar = model.n_similarity(test_files[i],test_files[k])
    if similar > threshold:
      Source.append(file[i][0])
      Target.append(file[k][0])
      Weight.append(similar) 

working on 0


NameError: ignored

In [0]:
for i in range(number_of_doc // 2, number_of_doc):
  print("working on " + str(i))
  for k in range(i+1, number_of_doc):    
    similar = model.n_similarity(test_files[i],test_files[k])
    if similar > threshold:
      Source.append(file[i][0])
      Target.append(file[k][0])
      Weight.append(similar) 

In [0]:
result = pd.DataFrame({'Source':Source, 'Target':Target, 'Similarity':Weight})

In [0]:
result

Important Note: To check each possible edge, there are around 500000 checks. Each check takes a second, it will cost us around 5 days to check all the edges.

In [0]:
from google.colab import drive
drive.mount('drive')

In [0]:
result.to_csv('Doc2Vec_edge_list.csv', index=False)
!cp Doc2Vec_edge_list.csv drive/My\ Drive/AWCA/Collab_notebooks/edge_list_creation/